In [1]:
from pathlib import Path
from image_captioning_with_blip.metrics import HuggingFaceEvaluator, load_ground_truth, load_predictions, huggingface_metrics
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

PREDICTIONS = Path("./captions.txt")
REFERENCE = Path("./mastani/Frames.json")


/mnt/local/ryan/.venv12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
predictions = load_predictions(PREDICTIONS)
ground_truths = load_ground_truth(REFERENCE)


In [3]:
df = pd.DataFrame({
    "predictions": predictions,
    "ground_truths": ground_truths,
})
df

,predictions,ground_truths
3347.jpg,a woman in a white dress dancing in a room,A person is dancing in a grand hall
6445.jpg,a scene from the movie the mummy mummy mummy m...,A person is carrying a tray with a lit lamp in...
5321.jpg,a scene from the movie ' ' ' ',Three people are dressed in traditional attire
328.jpg,a man in a white shirt and tie standing in fro...,a man looking away from camera
658.jpg,a scene from the movie ' s first trailer,a man in a helmet with a mustache and a large ...
...,...,...
5104.jpg,a woman standing in front of a building,A person is standing on a balcony of a grand b...
2740.jpg,a woman in a white dress standing in front of ...,a woman in a garment
4441.jpg,a scene from the movie ' s first trailer,A large group of people are dancing energetica...
515.jpg,a woman in a sari outfit with a gold nose ring,Priyanka with jewelry on her nose


In [4]:
df.sample(10)

,predictions,ground_truths
3211.jpg,a scene from the movie ' s trailer,Multiple people are performing a dance
1255.jpg,a group of knights standing in front of a dark...,a group of men in armor with swords
8029.jpg,a lone horse in a field under a stormy sky,A lone rider on a horse travels across a vast ...
7450.jpg,a man in a red hat and a woman in a white shirt,Two people are facing each other in a dimly li...
6042.jpg,a group of men standing in front of a temple,Three people are standing in front of ancient ...
119.jpg,a black and white image of a red object,a red and white background
3545.jpg,a scene from the movie ' ba ba ba ba ba ba ba ...,Multiple people are standing in a traditional ...
94.jpg,a car with the words ' i love you ' written on it,a blue sign with writing on it
7169.jpg,a woman in a red dress dancing with torches,A person is performing a dance
2500.jpg,a woman in a yellow sari outfit with a gold no...,Priyanka with her eyes closed


In [5]:
evaluator = HuggingFaceEvaluator(*huggingface_metrics)

[nltk_data] Downloading package wordnet to
[nltk_data]     /mnt/local/ryan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /mnt/local/ryan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /mnt/local/ryan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [38]:
evaluator.bertscore

EvaluationModule(name: "bert_score", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
BERTScore Metrics with the hashcode from a source against one or more references.

Args:
    predictions (list of str): Prediction/candidate sentences.
    references (list of str or list of list of str): Reference sentences.
    lang (str): Language of the sentences; required (e.g. 'en').
    model_type (str): Bert specification, default using the suggested
        model for the target language; has to specify at least one of
        `model_type` or `lang`.
    num_layers (int): The layer of representation to use,
        default using the number of layers tuned on WMT16 correlation data.
    verbose (bool): Turn on intermediate status update.
    

In [7]:
result = evaluator.evaluate(
    prediction=df.iloc[0]["predictions"],
    reference=df.iloc[0]["ground_truths"],
)
result

{'bleu': 0.0,
 'bleu_precisions': [0.3, 0.2222222222222222, 0.125, 0.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.25,
 'translation_length': 10,
 'reference_length': 8,
 'meteor': np.float64(0.45731707317073167),
 'rouge1': np.float64(0.4444444444444445),
 'rouge2': np.float64(0.25),
 'rougeL': np.float64(0.4444444444444445),
 'rougeLsum': np.float64(0.4444444444444445),
 'score': 15.619699684601283,
 'counts': [3, 2, 1, 0],
 'totals': [10, 9, 8, 7],
 'sacrebleu_precisions': [30.0, 22.22222222222222, 12.5, 7.142857142857143],
 'bp': 1.0,
 'sys_len': 10,
 'ref_len': 8,
 'exact_match': np.float64(0.0)}

In [10]:
r1 = evaluator.evaluator.compute(
    [df.iloc[0]["predictions"]],
    [df.iloc[0]["ground_truths"]],
)
r2 = evaluator.bertscore.compute(
    predictions=[df.iloc[0]["predictions"]],
    references=[df.iloc[0]["ground_truths"]],
    lang='en',
    device='cuda:3',
)
r1, r2

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


({'bleu': 0.0,
  'bleu_precisions': [0.3, 0.2222222222222222, 0.125, 0.0],
  'brevity_penalty': 1.0,
  'length_ratio': 1.25,
  'translation_length': 10,
  'reference_length': 8,
  'meteor': np.float64(0.45731707317073167),
  'rouge1': np.float64(0.4444444444444445),
  'rouge2': np.float64(0.25),
  'rougeL': np.float64(0.4444444444444445),
  'rougeLsum': np.float64(0.4444444444444445),
  'score': 15.619699684601283,
  'counts': [3, 2, 1, 0],
  'totals': [10, 9, 8, 7],
  'sacrebleu_precisions': [30.0, 22.22222222222222, 12.5, 7.142857142857143],
  'bp': 1.0,
  'sys_len': 10,
  'ref_len': 8,
  'exact_match': np.float64(0.0)},
 {'precision': [0.9028136730194092],
  'recall': [0.9177553653717041],
  'f1': [0.9102231860160828],
  'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.51.1)'})

In [11]:
r1|r2

{'bleu': 0.0,
 'bleu_precisions': [0.3, 0.2222222222222222, 0.125, 0.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.25,
 'translation_length': 10,
 'reference_length': 8,
 'meteor': np.float64(0.45731707317073167),
 'rouge1': np.float64(0.4444444444444445),
 'rouge2': np.float64(0.25),
 'rougeL': np.float64(0.4444444444444445),
 'rougeLsum': np.float64(0.4444444444444445),
 'score': 15.619699684601283,
 'counts': [3, 2, 1, 0],
 'totals': [10, 9, 8, 7],
 'sacrebleu_precisions': [30.0, 22.22222222222222, 12.5, 7.142857142857143],
 'bp': 1.0,
 'sys_len': 10,
 'ref_len': 8,
 'exact_match': np.float64(0.0),
 'precision': [0.9028136730194092],
 'recall': [0.9177553653717041],
 'f1': [0.9102231860160828],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.51.1)'}

In [14]:
bool(evaluator.evaluator.evaluation_modules[0])

False

In [29]:
def evaluate(row):
    pred = row["predictions"]
    gt = row["ground_truths"]
    return evaluator.evaluate(pred, gt)

results = df.apply(
    evaluate,
    axis=1,
    result_type="expand",
)

In [30]:
results

,bleu,bleu_precisions,brevity_penalty,length_ratio,translation_length,reference_length,meteor,rouge1,rouge2,rougeL,rougeLsum,score,counts,totals,sacrebleu_precisions,bp,sys_len,ref_len,exact_match
3347.jpg,0.000000,"[0.3, 0.2222222222222222, 0.125, 0.0]",1.000000,1.250000,10,8,0.457317,0.444444,0.250000,0.444444,0.444444,15.619700,"[3, 2, 1, 0]","[10, 9, 8, 7]","[30.0, 22.22222222222222, 12.5, 7.142857142857...",1.000000,10,8,0.0
6445.jpg,0.000000,"[0.05, 0.0, 0.0, 0.0]",1.000000,1.333333,20,15,0.032258,0.057143,0.000000,0.057143,0.057143,1.914603,"[1, 0, 0, 0]","[20, 19, 18, 17]","[5.0, 2.6315789473684212, 1.3888888888888888, ...",1.000000,20,15,0.0
5321.jpg,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.000000,1.285714,9,7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"[0, 0, 0, 0]","[9, 8, 7, 6]","[0.0, 0.0, 0.0, 0.0]",1.000000,9,7,0.0
328.jpg,0.000000,"[0.14285714285714285, 0.07692307692307693, 0.0...",1.000000,2.333333,14,6,0.220588,0.200000,0.111111,0.200000,0.200000,5.679677,"[2, 1, 0, 0]","[14, 13, 12, 11]","[14.285714285714286, 7.6923076923076925, 4.166...",1.000000,14,6,0.0
658.jpg,0.000000,"[0.2222222222222222, 0.0, 0.0, 0.0]",0.411112,0.529412,9,17,0.061728,0.160000,0.000000,0.160000,0.160000,2.330921,"[2, 0, 0, 0]","[9, 8, 7, 6]","[22.22222222222222, 6.25, 3.5714285714285716, ...",0.411112,9,17,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104.jpg,0.000000,"[0.625, 0.14285714285714285, 0.0, 0.0]",0.687289,0.727273,8,11,0.347664,0.526316,0.117647,0.526316,0.526316,9.545139,"[5, 1, 0, 0]","[8, 7, 6, 5]","[62.5, 14.285714285714286, 8.333333333333334, ...",0.687289,8,11,0.0
2740.jpg,0.212006,"[0.3333333333333333, 0.2727272727272727, 0.2, ...",1.000000,2.400000,12,5,0.438596,0.470588,0.400000,0.470588,0.470588,21.200627,"[4, 3, 2, 1]","[12, 11, 10, 9]","[33.333333333333336, 27.272727272727273, 20.0,...",1.000000,12,5,0.0
4441.jpg,0.000000,"[0.0, 0.0, 0.0, 0.0]",1.000000,1.125000,9,8,0.061728,0.125000,0.000000,0.125000,0.125000,0.000000,"[0, 0, 0, 0]","[9, 8, 7, 6]","[0.0, 0.0, 0.0, 0.0]",1.000000,9,8,0.0
515.jpg,0.000000,"[0.18181818181818182, 0.0, 0.0, 0.0]",1.000000,1.833333,11,6,0.153846,0.235294,0.000000,0.235294,0.235294,4.456883,"[2, 0, 0, 0]","[11, 10, 9, 8]","[18.181818181818183, 5.0, 2.7777777777777777, ...",1.000000,11,6,0.0


In [34]:
results.to_csv("evaluation_results.tsv", sep="\t", index=True)

In [31]:
df

,predictions,ground_truths
3347.jpg,a woman in a white dress dancing in a room,A person is dancing in a grand hall
6445.jpg,a scene from the movie the mummy mummy mummy m...,A person is carrying a tray with a lit lamp in...
5321.jpg,a scene from the movie ' ' ' ',Three people are dressed in traditional attire
328.jpg,a man in a white shirt and tie standing in fro...,a man looking away from camera
658.jpg,a scene from the movie ' s first trailer,a man in a helmet with a mustache and a large ...
...,...,...
5104.jpg,a woman standing in front of a building,A person is standing on a balcony of a grand b...
2740.jpg,a woman in a white dress standing in front of ...,a woman in a garment
4441.jpg,a scene from the movie ' s first trailer,A large group of people are dancing energetica...
515.jpg,a woman in a sari outfit with a gold nose ring,Priyanka with jewelry on her nose
